In [1]:
import requests
import json
import pandas as pd
import datetime
from dateutil.relativedelta import *

In [2]:
with open('key_info.json') as json_file:
    json_data = json.load(json_file)
    key = json_data['key']

In [3]:
df = pd.read_csv('movie_data.csv')['movieCd']

In [4]:
def get_movie_info_string(data, info_type, info_detail):
    result = []
    for i in range(len(data[info_type])):
        result.append(data[info_type][i][info_detail])
    result = ','.join(result)
    return result

In [5]:
data_list = []
for movieCd in df:
    data_dict = {}
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key='+key+'&movieCd='+str(movieCd)
    data = json.loads(requests.get(url).text)['movieInfoResult']['movieInfo']
    data_dict['movieCd']      = [data['movieCd']]
    data_dict['movieNmEn']    = [data['movieNmEn']]
    data_dict['showTm']       = [data['showTm']]
    data_dict['typeNm']       = [data['typeNm']]
    data_dict['nationNm']     = [get_movie_info_string(data, 'nations', 'nationNm')]
    data_dict['genreNm']      = [get_movie_info_string(data, 'genres', 'genreNm')]
    data_dict['directors']    = [get_movie_info_string(data, 'directors', 'peopleNm')]
    data_dict['actors']       = [get_movie_info_string(data, 'actors', 'peopleNm')]
    data_dict['watchGradeNm'] = [get_movie_info_string(data, 'audits', 'watchGradeNm')]
    data_df = pd.DataFrame(data_dict)
    data_list.append(data_df)

In [6]:
all_df = pd.concat(data_list).reset_index(drop=True)
all_df.head()

,movieCd,movieNmEn,showTm,typeNm,nationNm,genreNm,directors,actors,watchGradeNm
0,20201122,Tenet,149,장편,미국,"액션,SF",크리스토퍼 놀란,"존 데이비드 워싱턴,로버트 패틴슨,엘리자베스 데비키,애런 존슨,마이클 케인,케네스 ...",12세이상관람가
1,20197922,DELIVER US FROM EVIL,107,장편,한국,"범죄,액션",홍원찬,"황정민,이정재,박정민",15세이상관람가
2,20199883,OK! MADAM,100,장편,한국,"코미디,액션",이철하,"엄정화,박성웅,이상윤,배정남,이선빈,조성희,최진호,유수정",15세이상관람가
3,20202281,Crayon Shin-chan: Honeymoon Hurricane ~The Los...,99,장편,일본,애니메이션,하시모토 마사카즈,"박영남,김환진,강희선,여민정,정유미",전체관람가
4,20010238,Memento,113,장편,미국,"미스터리,범죄,스릴러",크리스토퍼 놀란,"가이 피어스,캐리 앤 모스,조 판톨리아노","15세관람가,15세이상관람가"


In [7]:
all_df.to_csv('movie_info_data.csv', index=False)